<a href="https://colab.research.google.com/github/igedeaguss/Magang-Pharos/blob/main/Uji%20Coba%20Kemiripan%20Produk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
import pandas as pd
import numpy as np
!pip install rapidfuzz
import warnings
warnings.filterwarnings("ignore")

# Load Dataset

In [2]:
data_ims = pd.read_excel('/content/drive/MyDrive/Dataset untuk sistem rekomendasi/IMS Compiled Q3 2019 - Q2 2024 (List Product).xlsx', sheet_name='Sheet1')
data_survey = pd.read_excel('/content/drive/MyDrive/Dataset untuk sistem rekomendasi/Data Hospital Survey Product Name Pivot.xlsx', sheet_name='Sheet1')

data_prdk_survey = data_survey.copy()
data_ims_prdk = data_ims.copy()

In [ ]:
data_ims_prdk

,Company1,Pack,Branded Details,Product,ATC1,ATC2,ATC3,ATC4,Pack Molecule String
0,ABAD DUA SATU MAK,HARVEST COUGH SYR 150ML,Branded Generic,HARVEST COUGH - ADD,R - RESPIRATORY SYSTEM,R05 - COUGH & COLD PREPARATIONS,R05F - OTHER COUGH & COLD PREPS,R05F0 - OTHER COUGH & COLD PREPS,CORDYCEPS SINENSIS SACC+ERIOBOTRYA JAPONICA+FR...
1,ABAD DUA SATU MAK,HARVEST COUGH SYR 60ML,Branded Generic,HARVEST COUGH - ADD,R - RESPIRATORY SYSTEM,R05 - COUGH & COLD PREPARATIONS,R05F - OTHER COUGH & COLD PREPS,R05F0 - OTHER COUGH & COLD PREPS,CORDYCEPS SINENSIS SACC+ERIOBOTRYA JAPONICA+FR...
2,ABAD DUA SATU MAK,SQ ACIDOPHILUS VEGICAP 50 BTL,Branded Generic,SQ ACIDOPHILUS - ADD,A - ALIMENTARY T.& METABOLISM,A07 - INTESTINAL DISORDER PROD,A07F - ANTIDIARR MICRO-ORGANISMS,A07F0 - ANTIDIARR MICRO-ORGANISMS,CELLULOSE+LACTOBACILLUS ACIDOPHILUS+MAGNESIUM
3,ABAD DUA SATU MAK,SQ ALFALFA TAB 500MG 50,Branded Generic,SQ ALFALFA - ADD,V - VARIOUS,V03 - OTHER THERAP.PROD.,V03X - ALL OTH.THERAPEUTIC PRDS,V03X0 - ALL OTH.THERAPEUTIC PRDS,MEDICAGO SATIVA
4,ABAD DUA SATU MAK,SQ ALLERBLAZ CAP 30 BTL,Branded Generic,SQ ALLERBLAZ - ADD,R - RESPIRATORY SYSTEM,R06 - ANTIHISTAMINES SYSTEMIC,R06A - ANTIHISTAMINES SYSTEMIC,R06A0 - ANTIHISTAMINES SYSTEMIC,ALBIZIA JULIBRISSIN+PHYLLANTHUS EMBLICA+PIPER ...
...,...,...,...,...,...,...,...,...,...
20380,ZENITH,ZETAMOL DRP 60MG/ML 20 BOX,Branded Generic,ZETAMOL - ZEN,N - NERVOUS SYSTEM,N02 - ANALGESICS,N02B - NON-NARCOTIC ANALGESICS,N02B0 - NON-NARCOTIC ANALGESICS,PARACETAMOL
20381,ZENITH,ZETAMOL SYR 120MG/5ML 60ML,Branded Generic,ZETAMOL - ZEN,N - NERVOUS SYSTEM,N02 - ANALGESICS,N02B - NON-NARCOTIC ANALGESICS,N02B0 - NON-NARCOTIC ANALGESICS,PARACETAMOL
20382,ZENITH,ZETAMOL TAB 500MG 100 BTL,Branded Generic,ZETAMOL - ZEN,N - NERVOUS SYSTEM,N02 - ANALGESICS,N02B - NON-NARCOTIC ANALGESICS,N02B0 - NON-NARCOTIC ANALGESICS,PARACETAMOL
20383,ZENITH,ZETAMOL TAB 500MG 1000 TIN,Branded Generic,ZETAMOL - ZEN,N - NERVOUS SYSTEM,N02 - ANALGESICS,N02B - NON-NARCOTIC ANALGESICS,N02B0 - NON-NARCOTIC ANALGESICS,PARACETAMOL


# Testing

In [ ]:
import pandas as pd
import re
from rapidfuzz import fuzz

# Normalisasi nama di data IMS
normalization_map = {
    "concentrate": "conc",
    "suppositoria": ("supp", "suppos"),
    "polybag": "polybag",
    "ointment": "oint",
    "krim": "crm",
    "lotion": "lot",
    "expectorant": "exp",
    "drop" : 'drp',
    "kaplet": "cap",
    "caplets": "cap",
    "suppos": "supp",
    "suspensi": "susp",
    "suspension": "susp",
    "forte": "fort",
    "syrup": "syr",
    "powder": "pwd",
    "solution": "sol",
    "tablet": "tab",
    "cream": "crm",
    "capsule": "cap",
    "kapsul": "cap",
    "kapsule": "cap",
    "chewable": "chew",
    "aerosol": "aerosol",
    "injeksi": "inj",
    "ampule": "amp",
    "inj": "inj",
    "injection" : ('inj', "amp"),
    "infusion": "infusion",
    "infus": "infusion",
    "inf": "infusion",
    "salep": "oint"
}

# Fungsi normalisasi nama produk
def normalize_name(name, norm_map):
    if pd.isna(name):
        return ""
    name = name.lower()
    for full, short in norm_map.items():
        if isinstance(short, (list, tuple)):
            for s in short:
                name = re.sub(r"\b" + re.escape(full) + r"\b", s, name)
        else:
            name = re.sub(r"\b" + re.escape(full) + r"\b", short, name)
    name = re.sub(r'\s+', ' ', name).strip()
    return name

# Fungsi mengambil kata pertama
def ambil_kata_pertama(nama_produk):
    return nama_produk.split()[0].lower() if nama_produk else ""

# Fungsi mengekstrak angka dari nama produk
def extract_numbers(nama_produk):
    return re.findall(r'\d+', nama_produk)

# Fungsi mengekstrak bentuk sediaan
def ekstrak_bentuk_sediaan(nama_produk):
    for bentuk in normalization_map.values():
        if isinstance(bentuk, (list, tuple)):
            for b in bentuk:
                if re.search(rf'\b{b}\b', nama_produk, flags=re.IGNORECASE):
                    return b
        else:
            if re.search(rf'\b{bentuk}\b', nama_produk, flags=re.IGNORECASE):
                return bentuk
    return None

# Normalisasi nama produk di data Survey
data_prdk_survey["normalized_name"] = data_prdk_survey["Product Name"].apply(lambda x: normalize_name(x, normalization_map))

# Normalisasi nama produk di data IMS
data_ims_prdk["normalized_name"] = data_ims_prdk["Pack"].apply(lambda x: normalize_name(x, normalization_map))

# Ambil kata pertama dari masing-masing dataset
data_prdk_survey['Kata Pertama'] = data_prdk_survey['Product Name'].apply(lambda x: ambil_kata_pertama(x))
data_ims_prdk['Kata Pertama'] = data_ims_prdk['Pack'].apply(lambda x: ambil_kata_pertama(x))

# List untuk menyimpan hasil terbaik
hasil_terbaik = []

# Looping pencocokan produk
for index, row in data_prdk_survey.iterrows():
    nama_survey = row['Product Name']
    kode_survey = row['Kode']
    nama_survey_normalized = row['normalized_name']
    kata_pertama = row['Kata Pertama']
    bentuk_survey = ekstrak_bentuk_sediaan(nama_survey_normalized)
    angka_survey = extract_numbers(nama_survey_normalized)

    # Filter produk IMS berdasarkan kata pertama dalam lowercase
    produk_ims_terfilter = data_ims_prdk[data_ims_prdk['Kata Pertama'] == kata_pertama.lower()]

    kandidat_produk = []
    hanya_nama_saja = []

    for ims in produk_ims_terfilter.itertuples():
        nama_ims_normalized = ims.normalized_name
        bentuk_ims = ekstrak_bentuk_sediaan(nama_ims_normalized)
        angka_ims = extract_numbers(nama_ims_normalized)
        ims_code = ims.KODE
        ims_zat_aktif = getattr(ims, "_8", None)
        ims_nfc1 = getattr(ims, "NFC1", None)
        ims_nfc_code = getattr(ims, "_4", None)

        if bentuk_survey:  # Jika bentuk sediaan ada pada produk survey
            if bentuk_survey == bentuk_ims:
                angka_score = len(set(angka_survey) & set(angka_ims)) * 25
                fuzzy_score = fuzz.token_sort_ratio(nama_survey_normalized, nama_ims_normalized)
                final_score = (0.6 * fuzzy_score) + (0.4 * angka_score)
                kandidat_produk.append((final_score, ims.Pack, ims_code, ims_zat_aktif, ims_nfc1, ims_nfc_code))
        else:  # Jika produk survey tidak memiliki bentuk sediaan
            angka_score = len(set(angka_survey) & set(angka_ims)) * 25
            fuzzy_score = fuzz.token_sort_ratio(nama_survey_normalized, nama_ims_normalized)
            final_score = (0.6 * fuzzy_score) + (0.4 * angka_score)
            kandidat_produk.append((final_score, ims.Pack, ims_code, ims_zat_aktif, ims_nfc1, ims_nfc_code))

        # Jika hanya nama produk (tanpa bentuk sediaan dan angka) cocok, simpan zat aktifnya
        if not angka_survey and not bentuk_survey:
            hanya_nama_saja.append((ims.Pack, ims_nfc1, ims_nfc_code, ims_zat_aktif))

    if kandidat_produk:
        kandidat_produk.sort(reverse=True, key=lambda x: x[0])
        best_score, best_match, best_ims_code, best_zat_aktif, best_nfc1, best_nfc_code = kandidat_produk[0]

        hasil_terbaik.append({
            'Kode Produk (Survey)' : kode_survey,
            'Nama Produk (Survey)': nama_survey,
            'Kata Pertama': kata_pertama,
            'Kode IMS': best_ims_code if angka_survey else None,
            'Nama Produk (IMS)': best_match if angka_survey else None,
            'Pack Molecule String': best_zat_aktif if not angka_survey else None,
            'NFC1': best_nfc1 if not angka_survey else None,
            'NFC Code': best_nfc_code if not angka_survey else None,
            'Skor Kemiripan': best_score
        })

# Konversi hasil ke DataFrame
data_hasil_final = pd.DataFrame(hasil_terbaik)

# Tampilkan DataFrame hasil
data_hasil_final

,Kode Produk (Survey),Nama Produk (Survey),Kata Pertama,Kode IMS,Nama Produk (IMS),Pack Molecule String,NFC1,NFC Code,Skor Kemiripan
0,1,ATORVASTATIN 20MG TAB/AMAROX,atorvastatin,1161.0,ATORVASTATIN TAB FLM CTD 20MG 30,None,None,None,53.8
1,2,BISOPROLOL 5 MG /HJ,bisoprolol,1920.0,BISOPROLOL TAB 5MG 30,None,None,None,50.2
2,3,BISOPROLOL 5 MG TABLET,bisoprolol,1920.0,BISOPROLOL TAB 5MG 30,None,None,None,58.0
3,5,CAPTOPRIL 12.5 MG,captopril,2480.0,CAPTOPRIL TAB 12.5MG 50,None,None,None,65.0
4,92,ABACAVIR 120MG /3TC 60MG TAB (DNS),abacavir,29.0,ABACAVIR SULF TAB FLM CTD 300MG 60 BTL,None,None,None,48.4
...,...,...,...,...,...,...,...,...,...
24056,50326,ZYPREXA ZYDIS 5MG TAB,zyprexa,18672.0,ZYPREXA TAB ZYDIS 5MG 28,None,None,None,65.8
24057,50330,ZYVOX 600 MG TABLET (LINEZOLID),zyvox,18677.0,ZYVOX TAB 600MG 10,None,None,None,50.8
24058,50332,ZYVOX 600MG TAB,zyvox,18677.0,ZYVOX TAB 600MG 10,None,None,None,64.6
24059,50333,ZYVOX 600MG/300ML POLYBAG,zyvox,18676.0,ZYVOX POLYBAG 2MG/ML 300ML,None,None,None,59.2


In [ ]:
for item in kandidat_produk:
    print(item)

(62.8, 'ZYVOX TAB 600MG 10', 18677, 'LINEZOLID', 'A - ORAL SOLID ORDINARY', 'A')
(62.8, 'ZYVOX TAB 600MG 10', 18677, 'LINEZOLID', 'A - ORAL SOLID ORDINARY', 'A')


## Find Pack Molecule

In [85]:
import re
import pandas as pd

def clean_product_name(name):
    return name.split(' -')[0].lower()

def clean_name_survey(name):
  # Hapus bagian setelah tanda "-" jika diikuti oleh karakter lain (misalnya "HARVEST COUGH - ADD" -> "HARVEST COUGH")
    name = re.sub(r"\s*-.*$", "", name)

    # Hapus karakter khusus tetapi tetap menjaga format yang benar
    name = re.sub(r'[\[\]{}@]', '', name)  # Hapus tanda [], {}, dan @
    name = re.sub(r'(?<!\s)-(?<!\s)', ' ', name)  # Ganti "-" yang tidak diapit spasi dengan spasi
    name = re.sub(r'[()]', '', name)  # Hapus tanda kurung biasa tetapi biarkan isinya

    # Hilangkan spasi ganda yang mungkin muncul akibat penghapusan
    name = re.sub(r'\s+', ' ', name).strip()

    return name.lower()

data_ims_prdk['Cleaned Brand Name'] = data_ims_prdk['Product'].apply(clean_product_name)
data_prdk_survey['Cleaned Name'] = data_prdk_survey['Product Name'].apply(clean_name_survey)

In [86]:
data_ims_sample = data_ims_prdk.sample(100)
data_survey_sample = data_prdk_survey.sample(200)

In [94]:
import pandas as pd
import re
from rapidfuzz import fuzz, process

def extract_dosage_form(product_name):
    dosage_form_map = {
        r'\bTabs?\b|\bTablets?\b': 'Tablet',
        r'\bCaps?\b|\bCapsules?\b|\bCapsul\b': 'Capsule',
        r'\bKapsul\b|\bKapsule\b|\bKap\b': 'Capsule',
        r'\bSuspensi\b|\bSuspension\b|\bSusp\b': 'Suspension',
        r'\bSerbuk\b|\bBubuk\b|\bPowder\b|\bPwd\b': 'Powder',
        r'\bOint\b|\bOintment\b|\bSalep\b': 'Ointment',
        r'\bKaplet\b|\bCaplets?\b|\bKapl\b|\bCaplet\b': 'Caplet',
        r'\bInj\b|\bInjeksi\b|\bInjection\b': 'Injection',
        r'\bAmp\b|\bAmpule\b|\bAmpul\b': 'Ampoule',
        r'\bCream\b|\bCrm\b|\bKrim\b|\bCr\b': 'Cream',
        r'\bSupp\b|\bSuppositoria\b|\bSuppos\b': 'Suppositoria',
        r'\bInf\b|\bInfusion\b|\bInfus\b': 'Infusion',
        r'\bSyrup\b|\bSirup\b|\bSyr\b': 'Syrup',
        r'\bDrop\b|\bDrops\b|\bTetes\b': 'Drop',
        r'\bConc\b|\bConcentrate\b': 'Concentrate',
        r'\bSol\b|\bSolution\b': 'Solution',
        r'\bGel\b': 'Gel'
    }
    for pattern, standard_form in dosage_form_map.items():
        if re.search(pattern, product_name, flags=re.IGNORECASE):
            return standard_form
    return ''

def filter_and_match_products(ims, survey):
    results = []
    processed_brands = set()

    for brand_keyword in ims['Cleaned Brand Name'].unique():
        if brand_keyword in processed_brands:
            continue

        processed_brands.add(brand_keyword)

        # Filter IMS berdasarkan Cleaned Brand Name
        filtered_ims = ims[ims['Cleaned Brand Name'].str.contains(fr"\b{re.escape(brand_keyword)}\b", case=False, na=False)]

        # Filter Survey berdasarkan Brand di IMS
        filtered_survey = survey[survey['Product Name'].str.contains(fr"\b{re.escape(brand_keyword)}\b", case=False, na=False)]

        for _, survey_row in filtered_survey.iterrows():
            survey_product = survey_row['Product Name']
            survey_form = survey_row['Dosage Form']

            # Filter IMS berdasarkan bentuk sediaan jika ada
            if survey_form:
                filtered_ims_form = filtered_ims[filtered_ims['Dosage Form'] == survey_form]
                if filtered_ims_form.empty:
                    filtered_ims_form = filtered_ims  # Jika tidak ada yang cocok, gunakan semua data yang sudah difilter berdasarkan brand
            else:
                filtered_ims_form = filtered_ims  # Tidak ada bentuk sediaan, gunakan semua data yang sudah difilter berdasarkan brand

            ims_pack_list = filtered_ims_form['Pack'].tolist()

            # Ambil semua kemungkinan kecocokan
            matches = process.extract(survey_product, ims_pack_list, scorer=fuzz.partial_token_sort_ratio, limit=len(ims_pack_list))

            if matches:
                best_match, best_score, idx = max(matches, key=lambda x: x[1])
                matched_row = filtered_ims_form.iloc[idx]

                result_entry = {
                    'Survey Product': survey_product,
                    'Match IMS': best_match,
                    'Similarity Score': best_score,
                    'Pack Molecule String': matched_row['Pack Molecule String'],
                    'Brand': matched_row['Cleaned Brand Name']
                }

                if result_entry not in results:
                    results.append(result_entry)

    return pd.DataFrame(results)

# Pra-pemrosesan data
data_ims_prdk['Dosage Form'] = data_ims_prdk['Pack'].apply(extract_dosage_form)
data_prdk_survey['Dosage Form'] = data_prdk_survey['Product Name'].apply(extract_dosage_form)

# Contoh penggunaan
filtered_results = filter_and_match_products(data_ims_prdk, data_prdk_survey)
filtered_results

,Survey Product,Match IMS,Similarity Score,Pack Molecule String,Brand
0,ABBOTIC XL ( OD ) TAB,ABBOTIC TAB XL 500MG 10,83.333333,CLARITHROMYCIN,abbotic
1,ABBOTIC 125 MG / 30 ML,ABBOTIC SYR DRY 125MG/5ML 30ML,68.181818,CLARITHROMYCIN,abbotic
2,ABBOTIC 125 MG / 60 ML,ABBOTIC SYR DRY 125MG/5ML 60ML,68.181818,CLARITHROMYCIN,abbotic
3,ABBOTIC 125 MG/5 ML 30 ML DRY SIRUP,ABBOTIC SYR DRY 125MG/5ML 30ML,77.551020,CLARITHROMYCIN,abbotic
4,ABBOTIC 125 MG/5 ML 60 ML DRY SYRUP,ABBOTIC SYR DRY 125MG/5ML 60ML,77.551020,CLARITHROMYCIN,abbotic
...,...,...,...,...,...
37604,SPEDIFEN 400 TABLET,SPEDIFEN TAB FLM CTD 400MG 30,81.250000,IBUPROFEN,spedifen
37605,SPEDIFEN 400MG TAB,SPEDIFEN TAB FLM CTD 400MG 30,87.500000,IBUPROFEN,spedifen
37606,SPEDIFEN TAB 400 MG400,SPEDIFEN TAB FLM CTD 400MG 30,77.777778,IBUPROFEN,spedifen
37607,"Spedifen Tab, 400 mg",SPEDIFEN TAB FLM CTD 400MG 30,36.363636,IBUPROFEN,spedifen


In [95]:
import os
import datetime

# Tentukan nama file berdasarkan tanggal saat ini
today_date = datetime.datetime.now().strftime('%d%m%Y')  # Format tanggal ddmmyyyy
file_name = f"{today_date} Data Hasil Match Product (IMS & Survey) V2.xlsx"  # Format nama file

# Tentukan direktori penyimpanan
save_directory = "/content/drive/MyDrive/Dataset Hasil Match Product"
file_path = os.path.join(save_directory, file_name)

# Simpan DataFrame ke file Excel
filtered_results.to_excel(file_path, index=False)

print(f"File berhasil disimpan di: {file_path}")

File berhasil disimpan di: /content/drive/MyDrive/Dataset Hasil Match Product/19022025 Data Hasil Match Product (IMS & Survey) V2.xlsx


In [32]:
# Cara kerja sistem pencocokannya
from rapidfuzz import fuzz, process


a = ["ABBOTIC SYR DRY 125MG/5ML 30ML", "ABBOTIC SYR DRY 125MG/5ML 60ML", "ABBOTIC SYR DRY 250MG/5ML 50ML",
     "ABBOTIC TAB FLM CTD 250MG 100 BOX","ABBOTIC TAB FLM CTD 500MG 30","ABBOTIC TAB XL 500MG 10"]

b = "ABBOTIC 250MG SYR 50 ML" #asumsi data survey yang ingin dicari kecocokannya

for i in range(len(a)):
  score = fuzz.partial_token_sort_ratio(a[i], b)
  print(f"{b} | {a[i]} | {score:.2f}")

ABBOTIC 250MG SYR 50 ML | ABBOTIC SYR DRY 125MG/5ML 30ML | 71.43
ABBOTIC 250MG SYR 50 ML | ABBOTIC SYR DRY 125MG/5ML 60ML | 71.43
ABBOTIC 250MG SYR 50 ML | ABBOTIC SYR DRY 250MG/5ML 50ML | 74.42
ABBOTIC 250MG SYR 50 ML | ABBOTIC TAB FLM CTD 250MG 100 BOX | 68.29
ABBOTIC 250MG SYR 50 ML | ABBOTIC TAB FLM CTD 500MG 30 | 65.22
ABBOTIC 250MG SYR 50 ML | ABBOTIC TAB XL 500MG 10 | 66.67


In [97]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Input dari pengguna
brand = 'Abbotic'
produk_survey = 'ABBOTIC 500MG TAB'

# Fungsi untuk normalisasi teks
def normalize_text(text):
    text = text.lower()  # Ubah menjadi huruf kecil
    text = re.sub(r'[^\w\s]', '', text)  # Hapus karakter khusus
    return text

# Subset data berdasarkan brand
subset_data = data_ims_prdk[data_ims_prdk['Cleaned Brand Name'].str.contains(brand, case=False)]
subset_pack = subset_data[['Pack']]

# Menghitung skor kemiripan
def calculate_combined_similarity(produk_survey, pack_list):
    similarity_scores = []
    normalized_survey = normalize_text(produk_survey)
    for pack in pack_list:
        normalized_pack = normalize_text(pack)

        # Hitung skor menggunakan kedua metode
        #score_token_set = fuzz.token_set_ratio(normalized_survey, normalized_pack)
        score_partial_token_sort = fuzz.partial_token_sort_ratio(normalized_survey, normalized_pack)

        # Gabungkan skor (misalnya, rata-rata)
        #combined_score = (score_token_set + score_partial_token_sort) / 2
        similarity_scores.append(score_partial_token_sort)

    return similarity_scores


# Mendapatkan daftar pack
pack_list = subset_pack['Pack'].tolist()

# Menghitung skor kemiripan
similarity_scores = calculate_combined_similarity(produk_survey, pack_list)

subset_pack['Product Survey'] = produk_survey

# Menambahkan skor kemiripan ke dalam DataFrame
subset_pack['Skor Kemiripan'] = similarity_scores

# Menampilkan hasil
subset_pack


,Pack,Product Survey,Skor Kemiripan
76,ABBOTIC SYR DRY 125MG/5ML 30ML,ABBOTIC 500MG TAB,70.588235
77,ABBOTIC SYR DRY 125MG/5ML 60ML,ABBOTIC 500MG TAB,70.588235
78,ABBOTIC SYR DRY 250MG/5ML 50ML,ABBOTIC 500MG TAB,70.588235
79,ABBOTIC TAB FLM CTD 250MG 100 BOX,ABBOTIC 500MG TAB,82.352941
80,ABBOTIC TAB FLM CTD 500MG 30,ABBOTIC 500MG TAB,88.235294
81,ABBOTIC TAB XL 500MG 10,ABBOTIC 500MG TAB,100.000000


## Mapping Dosage Form

In [ ]:
def extract_dosage_form(product_name):
    # Kamus untuk standarisasi bentuk sediaan
    dosage_form_map = {
        r'\bTabs?\b|\bTablets?\b': 'Tablet',
        r'\bCaps?\b|\bCapsules?\b|\bCapsul\b': 'Capsule',
        r'\bKapsul\b|\bKapsule\b|\bKap\b': 'Capsule',
        r'\bSuspensi\b|\bSuspension\b|\bSusp\b': 'Suspension',
        r'\bSerbuk\b|\bBubuk\b|\bPowder\b|\bPwd\b': 'Powder',
        r'\bOint\b|\bOintment\b|\bSalep\b': 'Ointment',
        r'\bKaplet\b|\bCaplets?\b|\bKapl\b|\bCaplet\b': 'Caplet',
        r'\bInj\b|\bInjeksi\b|\bInjection\b': 'Injection',
        r'\bAmp\b|\bAmpule\b|\bAmpul\b': 'Ampoule',
        r'\bCream\b|\bCrm\b|\bKrim\b|\bCr\b': 'Cream',
        r'\bSupp\b|\bSuppositoria\b|\bSuppos\b': 'Suppositoria',
        r'\bInf\b|\bInfusion\b|\bInfus\b': 'Infusion',
        r'\bSyrup\b|\bSirup\b|\bSyr\b': 'Syrup',
        r'\bDrop\b|\bDrops\b|\bTetes\b': 'Drop',
        r'\bConc\b|\bConcentrate\b': 'Concentrate',
        r'\bSol\b|\bSolution\b': 'Solution',
        r'\bGel\b': 'Gel'
    }

    # Iterasi melalui kamus dan cari bentuk sediaan
    for pattern, standard_form in dosage_form_map.items():
        if re.search(pattern, product_name, flags=re.IGNORECASE):
            return standard_form

    return ''  # Kembalikan string kosong jika tidak ada bentuk sediaan yang cocok

def create_standardized_dataframe(data):
    # Ekstrak bentuk sediaan yang distandarisasi
    data['Dosage Form'] = data['Product Name'].apply(extract_dosage_form)

    # Pilih kolom yang diinginkan
    return data[['Product Name', 'Dosage Form']]

standardized_df = create_standardized_dataframe(data_prdk_survey.copy())
standardized_df

,Product Name,Dosage Form
0,ATORVASTATIN 20MG TAB/AMAROX,Tablet
1,BISOPROLOL 5 MG /HJ,
2,BISOPROLOL 5 MG TABLET,Tablet
3,CALOS KAPLET,Caplet
4,CAPTOPRIL 12.5 MG,
...,...,...
50330,ZYVOX 600MG INJ 300ML,Injection
50331,ZYVOX 600MG TAB,Tablet
50332,ZYVOX 600MG/300ML POLYBAG,
50333,Zyvox Infusion 2 mg/mL - 300 mL,Infusion


In [ ]:
standardized_df[standardized_df['Product Name'] == 'XYLOCAIN 2% GEL SYRINGE 10G']

,Product Name,Dosage Form
49783,XYLOCAIN 2% GEL SYRINGE 10G,Gel


In [ ]:
import os
import datetime

# Tentukan nama file berdasarkan tanggal saat ini
today_date = datetime.datetime.now().strftime('%d%m%Y')  # Format tanggal ddmmyyyy
file_name = f"{today_date} Mapping bentuk sediaan survey.xlsx"  # Format nama file

# Tentukan direktori penyimpanan
save_directory = "/content/drive/MyDrive/Dataset Hasil Match Product"
file_path = os.path.join(save_directory, file_name)

# Simpan DataFrame ke file Excel
standardized_df.to_excel(file_path, index=False)

print(f"File berhasil disimpan di: {file_path}")

File berhasil disimpan di: /content/drive/MyDrive/Dataset Hasil Match Product/18022025 Mapping bentuk sediaan survey.xlsx


# Fixed Algorithm (Use This)

In [ ]:
import pandas as pd
import re
from fuzzywuzzy import fuzz

# Normalisasi nama di data IMS
normalization_map = {
    "concentrate": "conc",
    "suppositoria": ("supp", "suppos"),
    "polybag": "polybag",
    "ointment": "oint",
    "krim": "crm",
    "lotion": "lot",
    "expectorant": "exp",
    "drop": "drp",
    "kaplet": "cap",
    "caplets": "cap",
    "suppos": "supp",
    "suspensi": "susp",
    "suspension": "susp",
    "forte": "fort",
    "syrup": "syr",
    "powder": "pwd",
    "solution": "sol",
    "tablet": "tab",
    "cream": "crm",
    "capsule": "cap",
    "kapsul": "cap",
    "kapsule": "cap",
    "chewable": "chew",
    "aerosol": "aerosol",
    "injeksi": "inj",
    "ampule": "amp",
    "inj": "inj",
    "injection": ("inj", "amp"),
    "infusion": "infusion",
    "infus": "infusion",
    "inf": "infusion",
    "salep": "oint"
}

# Fungsi normalisasi nama produk
def normalize_name(name, norm_map):
    if pd.isna(name):
        return ""

    name = name.lower()

    # Normalisasi bentuk sediaan
    for full, short in norm_map.items():
        if isinstance(short, (list, tuple)):
            for s in short:
                name = re.sub(r"\b" + re.escape(full) + r"\b", s, name)
        else:
            name = re.sub(r"\b" + re.escape(full) + r"\b", short, name)

    # Hapus karakter khusus tetapi tetap menjaga format yang benar
    name = re.sub(r'[\[\]{}@]', '', name)  # Hapus tanda [], {}, dan @
    name = re.sub(r'(?<!\s)-(?<!\s)', ' ', name)  # Ganti "-" yang tidak diapit spasi dengan spasi
    name = re.sub(r'[()]', '', name)  # Hapus tanda kurung biasa tetapi biarkan isinya

    # Hilangkan spasi ganda yang mungkin muncul akibat penghapusan
    name = re.sub(r'\s+', ' ', name).strip()

    return name

# Fungsi mengambil kata pertama
def ambil_kata_pertama(nama_produk):
    return nama_produk.split()[0].lower() if nama_produk else ""

# Fungsi mengekstrak angka dari nama produk
def extract_numbers(nama_produk):
    return re.findall(r'\d+', nama_produk)

# Fungsi mengekstrak bentuk sediaan
def ekstrak_bentuk_sediaan(nama_produk):
    for bentuk in normalization_map.values():
        if isinstance(bentuk, (list, tuple)):
            for b in bentuk:
                if re.search(rf'\b{b}\b', nama_produk, flags=re.IGNORECASE):
                    return b
        else:
            if re.search(rf'\b{bentuk}\b', nama_produk, flags=re.IGNORECASE):
                return bentuk
    return None

# Normalisasi nama produk di data Survey
data_prdk_survey["normalized_name"] = data_prdk_survey["Product Name"].apply(lambda x: normalize_name(x, normalization_map))

# Normalisasi nama produk di data IMS
data_ims_prdk["normalized_name"] = data_ims_prdk["Pack"].apply(lambda x: normalize_name(x, normalization_map))

# Ambil kata pertama dari masing-masing dataset
data_prdk_survey['Kata Pertama'] = data_prdk_survey['normalized_name'].apply(lambda x: ambil_kata_pertama(x))
data_ims_prdk['Kata Pertama'] = data_ims_prdk['Pack'].apply(lambda x: ambil_kata_pertama(x))

# List untuk menyimpan hasil terbaik
hasil_terbaik = []

# Looping pencocokan produk
for index, row in data_prdk_survey.iterrows():
    nama_survey = row['Product Name']
    kode_survey = row['Kode']
    nama_survey_normalized = row['normalized_name']
    kata_pertama = row['Kata Pertama']
    bentuk_survey = ekstrak_bentuk_sediaan(nama_survey_normalized)
    angka_survey = extract_numbers(nama_survey_normalized)

    # Filter produk IMS berdasarkan kata pertama dalam lowercase
    produk_ims_terfilter = data_ims_prdk[data_ims_prdk['Kata Pertama'] == kata_pertama.lower()]

    kandidat_produk = []

    for ims in produk_ims_terfilter.itertuples():
        nama_ims_normalized = ims.normalized_name
        bentuk_ims = ekstrak_bentuk_sediaan(nama_ims_normalized)
        angka_ims = extract_numbers(nama_ims_normalized)
        kata_pertama_ims = nama_ims_normalized.split()[0]  # Ambil kata pertama
        pack_molecule_ims = getattr(ims, "_8", None)  # Ambil Pack Molecule String dari IMS
        ims_nfc1 = getattr(ims, "NFC1", None)
        ims_nfc_code = getattr(ims, "_4", None)

        angka_score = len(set(angka_survey) & set(angka_ims)) * 25
        fuzzy_score = fuzz.token_sort_ratio(nama_survey_normalized, nama_ims_normalized)

        # Jika bentuk sediaan ada, tetapi tidak ada di IMS, buat null pada produk IMS
        if bentuk_survey and bentuk_ims != bentuk_survey:
            kandidat_produk.append((None, None, None, pack_molecule_ims, None, None))
        else:
            final_score = (0.6 * fuzzy_score) + (0.4 * angka_score)
            kandidat_produk.append((final_score, ims.Pack, ims.KODE, pack_molecule_ims, ims_nfc1, ims_nfc_code))

    # Simpan hasil pencocokan terbaik
    if kandidat_produk:
        kandidat_produk.sort(reverse=True, key=lambda x: (x[0] is not None, x[0]))
        best_score, best_match, best_ims_code, best_pack_molecule_ims, best_nfc1, best_nfc_code = kandidat_produk[0]

        hasil_terbaik.append({
            'Kode Produk (Survey)': kode_survey,
            'Nama Produk (Survey)': nama_survey,
            'Kata Pertama': kata_pertama,
            'Kode IMS': best_ims_code,
            'Nama Produk (IMS)': best_match,
            'Pack Molecule String (IMS)': best_pack_molecule_ims,
            "NFC1" : best_nfc1,
            "NFC Code" : best_nfc_code,
            'Skor Kemiripan': best_score
        })

# Konversi hasil ke DataFrame
data_hasil_final_1 = pd.DataFrame(hasil_terbaik)

# Tampilkan DataFrame hasil
data_hasil_final_1


,Kode Produk (Survey),Nama Produk (Survey),Kata Pertama,Kode IMS,Nama Produk (IMS),Pack Molecule String (IMS),NFC1,NFC Code,Skor Kemiripan
0,1,ATORVASTATIN 20MG TAB/AMAROX,atorvastatin,1161.0,ATORVASTATIN TAB FLM CTD 20MG 30,ATORVASTATIN,A - ORAL SOLID ORDINARY,A,53.8
1,2,BISOPROLOL 5 MG /HJ,bisoprolol,1920.0,BISOPROLOL TAB 5MG 30,BISOPROLOL,A - ORAL SOLID ORDINARY,A,50.2
2,3,BISOPROLOL 5 MG TABLET,bisoprolol,1920.0,BISOPROLOL TAB 5MG 30,BISOPROLOL,A - ORAL SOLID ORDINARY,A,58.0
3,4,CALOS KAPLET,calos,NaN,None,CALCIUM,None,None,NaN
4,5,CAPTOPRIL 12.5 MG,captopril,2480.0,CAPTOPRIL TAB 12.5MG 50,CAPTOPRIL,A - ORAL SOLID ORDINARY,A,65.0
...,...,...,...,...,...,...,...,...,...
31379,50331,ZYVOX 600MG INJ 300ML,zyvox,NaN,None,LINEZOLID,None,None,NaN
31380,50332,ZYVOX 600MG TAB,zyvox,18677.0,ZYVOX TAB 600MG 10,LINEZOLID,A - ORAL SOLID ORDINARY,A,64.6
31381,50333,ZYVOX 600MG/300ML POLYBAG,zyvox,18676.0,ZYVOX POLYBAG 2MG/ML 300ML,LINEZOLID,F - PARENTERAL ORDINARY,F,59.2
31382,50334,Zyvox Infusion 2 mg/mL - 300 mL,zyvox,NaN,None,LINEZOLID,None,None,NaN


In [ ]:
data_prdk_survey.head(10)

,Kode,Product Name,normalized_name,Kata Pertama
0,1,ATORVASTATIN 20MG TAB/AMAROX,atorvastatin 20mg tab/amarox,atorvastatin
1,2,BISOPROLOL 5 MG /HJ,bisoprolol 5 mg /hj,bisoprolol
2,3,BISOPROLOL 5 MG TABLET,bisoprolol 5 mg tab,bisoprolol
3,4,CALOS KAPLET,calos cap,calos
4,5,CAPTOPRIL 12.5 MG,captopril 12.5 mg,captopril
5,6,CEFOPERAZONE 1G INJ/BRN,cefoperazone 1g inj/brn,cefoperazone
6,7,MESS ONEMED 11,mess onemed 11,mess
7,8,PROMUS PREMIER SELECT 3.5 MM X 16 MM / DIPA,promus premier select 3.5 mm x 16 mm / dipa,promus
8,9,[ALBOTHYL CONCENTRATE 10 ML,albothyl conc 10 ml,albothyl
9,10,0.9% NACL 1 LTR ( B BRAUN),0.9% nacl 1 ltr b braun,0.9%


In [ ]:
data_hasil_final_1[data_hasil_final_1['Kata Pertama'].str.contains("albothyl", case=False, na=True)]

,Kode Produk (Survey),Nama Produk (Survey),Kata Pertama,Kode IMS,Nama Produk (IMS),Pack Molecule String (IMS),NFC1,NFC Code,Skor Kemiripan
6,9,[ALBOTHYL CONCENTRATE 10 ML,albothyl,297.0,ALBOTHYL SOL CONC 10ML,POLICRESULEN,K - ORAL TOPICAL,K,62.8
444,690,Albothyl Concentrate 10ml,albothyl,297.0,ALBOTHYL SOL CONC 10ML,POLICRESULEN,K - ORAL TOPICAL,K,64.0
445,691,ALBOTHYL 10 CC@DOS,albothyl,297.0,ALBOTHYL SOL CONC 10ML,POLICRESULEN,K - ORAL TOPICAL,K,56.2
446,692,ALBOTHYL 10 ML,albothyl,297.0,ALBOTHYL SOL CONC 10ML,POLICRESULEN,K - ORAL TOPICAL,K,53.2
447,693,ALBOTHYL 50GRAM VAG GEL,albothyl,300.0,ALBOTHYL VAG SUPP 90MG 6 BOX,POLICRESULEN,T - VAGINAL,T,37.8
448,694,ALBOTHYL CONCENTRATE 10 ML,albothyl,297.0,ALBOTHYL SOL CONC 10ML,POLICRESULEN,K - ORAL TOPICAL,K,62.8
449,695,Albothyl Concentrate 5ml,albothyl,299.0,ALBOTHYL SOL CONC 5ML,POLICRESULEN,K - ORAL TOPICAL,K,63.4
450,696,ALBOTHYL SOL 10 ML,albothyl,NaN,None,POLICRESULEN,None,None,NaN
451,697,ALBOTHYL SOL BTL 10 ML,albothyl,NaN,None,POLICRESULEN,None,None,NaN
452,698,ALBOTHYL VAGINAL TAB,albothyl,NaN,None,POLICRESULEN,None,None,NaN


In [ ]:
print(fuzz.token_sort_ratio("ACETYLCYSTEINE", "ACETYLCISTEIN"))

89


In [ ]:
#Melakukan join data yang match dengan yang tidak match
data_final = data_survey.merge(data_hasil_final_1[['Kode Produk (Survey)', 'Kode IMS','Nama Produk (IMS)','Pack Molecule String (IMS)','NFC1','NFC Code','Skor Kemiripan']],
                               left_on='Kode',
                               right_on="Kode Produk (Survey)",
                               how='left')

#Mengambil kolom yang relevan
data_final = data_final[["Kode","Product Name",'Kode IMS','Nama Produk (IMS)','Pack Molecule String (IMS)','NFC1','NFC Code','Skor Kemiripan']]
data_final.rename(columns = {"Product Name": "Nama Produk Survey"}, inplace=True)
data_final

,Kode,Nama Produk Survey,Kode IMS,Nama Produk (IMS),Pack Molecule String (IMS),NFC1,NFC Code,Skor Kemiripan
0,1,ATORVASTATIN 20MG TAB/AMAROX,1161.0,ATORVASTATIN TAB FLM CTD 20MG 30,ATORVASTATIN,A - ORAL SOLID ORDINARY,A,53.8
1,2,BISOPROLOL 5 MG /HJ,1920.0,BISOPROLOL TAB 5MG 30,BISOPROLOL,A - ORAL SOLID ORDINARY,A,50.2
2,3,BISOPROLOL 5 MG TABLET,1920.0,BISOPROLOL TAB 5MG 30,BISOPROLOL,A - ORAL SOLID ORDINARY,A,58.0
3,4,CALOS KAPLET,NaN,None,CALCIUM,None,None,NaN
4,5,CAPTOPRIL 12.5 MG,2480.0,CAPTOPRIL TAB 12.5MG 50,CAPTOPRIL,A - ORAL SOLID ORDINARY,A,65.0
...,...,...,...,...,...,...,...,...
50330,50331,ZYVOX 600MG INJ 300ML,NaN,None,LINEZOLID,None,None,NaN
50331,50332,ZYVOX 600MG TAB,18677.0,ZYVOX TAB 600MG 10,LINEZOLID,A - ORAL SOLID ORDINARY,A,64.6
50332,50333,ZYVOX 600MG/300ML POLYBAG,18676.0,ZYVOX POLYBAG 2MG/ML 300ML,LINEZOLID,F - PARENTERAL ORDINARY,F,59.2
50333,50334,Zyvox Infusion 2 mg/mL - 300 mL,NaN,None,LINEZOLID,None,None,NaN


In [ ]:
data_survey.head()

,Kode,Product Name
0,1,ATORVASTATIN 20MG TAB/AMAROX
1,2,BISOPROLOL 5 MG /HJ
2,3,BISOPROLOL 5 MG TABLET
3,4,CALOS KAPLET
4,5,CAPTOPRIL 12.5 MG


## Save to Gdrive

In [ ]:
import os
import datetime

# Tentukan nama file berdasarkan tanggal saat ini
today_date = datetime.datetime.now().strftime('%d%m%Y')  # Format tanggal ddmmyyyy
file_name = f"{today_date} Data Hasil Match Product (IMS & Survey).xlsx"  # Format nama file

# Tentukan direktori penyimpanan
save_directory = "/content/drive/MyDrive/Dataset Hasil Match Product"
file_path = os.path.join(save_directory, file_name)

# Simpan DataFrame ke file Excel
data_final.to_excel(file_path, index=False)

print(f"File berhasil disimpan di: {file_path}")

File berhasil disimpan di: /content/drive/MyDrive/Dataset Hasil Match Product/17022025 Data Hasil Match Product (IMS & Survey).xlsx
